In [0]:
import requests
import pandas as pd
from pyspark.sql import functions as F

In [0]:
indicators = [
    {"code": "SCSUD_MEDNTG", "name": "Cobertura de serviços, medicamento incluído em manual/t diretrizes nacionais", "short": "med_ntg"},
    {"code": "SCSUD_REHAB", "name": "Cobertura de serviços, programas de reabilitação", "short": "rehab"},
    {"code": "PRISON_D3_DEATHS_DRUG_MRATE", "name": "Taxa de mortalidade por overdose de drogas em prisões (por 100.000 presos/ano)", "short": "prison_overdose"},
    {"code": "SCSUD_GOVPROGRESS", "name": "Cobertura de serviços, progresso em governança", "short": "gov_progress"},
    {"code": "PRISON_B15_REPDRUGEVENTS", "name": "Processo padronizado para eventos adversos por drogas em prisões", "short": "prison_adverse_events"},
    {"code": "SUD_TREATMENTSERVICES_COVERAGE", "name": "Cobertura de serviços de tratamento para transtornos por uso de substâncias (%)", "short": "treatment_coverage"},
    {"code": "SCSUD_MEDREG", "name": "Cobertura de serviços, medicamento registrado no país", "short": "med_reg"},
    {"code": "SCSUD_FINANCINGEXP", "name": "Cobertura de serviços, informações sobre gastos governamentais", "short": "financing_exp"},
    {"code": "SUD_SERVICECAPACITYINDEX", "name": "Índice de Capacidade de Serviços para Transtornos por Uso de Substâncias (SCI-SUD)", "short": "capacity_index"},
    {"code": "SCSUD_MEDPUBLIC", "name": "Cobertura de serviços, medicamento fornecido gratuitamente no sistema público de saúde", "short": "med_public"},
    {"code": "SCSUD_REHABPROGRESS", "name": "Cobertura de serviços, progresso em programas de reabilitação", "short": "rehab_progress"},
    {"code": "SCSUD_NSS", "name": "Cobertura de serviços, serviços não estruturados", "short": "services_nss"},
    {"code": "HEPATITIS_HCV_TREATMENT_PWID_PERCENT", "name": "Pessoas vivendo com HIV que injetam drogas, tratadas para hepatite C crônica (%)", "short": "hcv_treated_pwid"},
    {"code": "SCSUD_PHARM", "name": "Cobertura de serviços, serviços de tratamento farmacológico", "short": "pharm"},
    {"code": "SCSUD_MEDDF", "name": "Cobertura de serviços, medicamento incluído em formulário de drogas", "short": "med_formulary"},
    {"code": "PWID_PSE_NUM", "name": "Pessoas que injetam drogas: estimativas populacionais (número)", "short": "pwid_pop"},
    {"code": "PRISON_D4_SUD_DIAG_TOT", "name": "Diagnósticos ativos de transtornos por drogas em prisões (número)", "short": "prison_sud_diag"},
    {"code": "SCSUD_MEDACCESSPROGRESS", "name": "Cobertura de serviços, progresso no acesso a medicamentos", "short": "med_access_progress"},
    {"code": "SCSUD_PHARMPROGRESS", "name": "Cobertura de serviços, progresso em serviços farmacológicos", "short": "pharm_progress"},
    {"code": "HEPATITIS_HCV_TESTING_PWID_PERCENT", "name": "Pessoas vivendo com HIV que injetam drogas, diagnosticadas com hepatite C crônica (%)", "short": "hcv_test_pwid"}
]


In [0]:
# Caminho base S3
landing_base = "s3://bucketbastet/data-suicide/drugs/"

# Loop para cada indicador
for ind in indicators:
    code = ind["code"]
    name = ind["name"]
    short = ind["short"]

    print(f"\n Processando indicador {code} - {name}")

    # 1) Download WHO API
    url = f"https://ghoapi.azureedge.net/api/{code}?$filter=date(TimeDimensionBegin) ge 2000-01-01"
    resp = requests.get(url, timeout=60)

    if resp.status_code != 200:
        print(f"Falha {resp.status_code} ao baixar {code}")
        continue

    data = resp.json().get("value", [])
    if not data:
        print(f"Nenhum dado retornado para {code}")
        continue

    # 2) Normaliza e cria Spark DF
    raw_df = pd.json_normalize(data)
    spark_df = spark.createDataFrame(raw_df)

    void_cols = [c for c, t in spark_df.dtypes if t == "void"]
    spark_df = spark_df.drop(*void_cols) if void_cols else spark_df
    spark_df = spark_df.withColumn("ingestion_ts", F.current_timestamp())

    # 3) Salvar Landing (Parquet por indicador)
    landing_path = f"{landing_base}{short}/"
    display(spark_df)

    # 4) Salvar Bronze
    bronze_table = f"bronze.data_suicide.drug_{short}"
    (
        spark_df.write
        .mode("overwrite")
        .option("mergeSchema", "true")
        .saveAsTable(bronze_table)
    )
    print(f"Bronze salvo em {bronze_table}")
    display(spark_df)
    # 5) Documentar tabela
    spark.sql(f"""
    COMMENT ON TABLE {bronze_table} IS
    'WHO GHO - {name} (IndicatorCode={code}). Fonte: {url}'
    """)
